## 미니배치 학습 알고리즘 구현

#### 헤더 및 함수 선언부

In [35]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

# 교차 엔트로피 오차 CEE(Cross Entropy Error)
# label의 값이 one_hot encoding일 때만 사용이 가능.
def cross_entropy_error(t, y):
    delta = 1e-7 # log 0을 막기위해    
    cost = -np.sum(t*np.log(y+delta))
    return cost

# 미니배치 교차 엔트로피 오차 CEE(Cross Entropy Error)
# 기존 CEE에서 행렬 처리랑 평균값 계산만 추가한거
def cross_entropy_error_avg(t, y):
    batch_size = y.shape[0]
    cost = 0
    if batch_size == 1:        
        return cross_entropy_error(t, y)
    else:
        for idx in range(0, batch_size):
            cost += cross_entropy_error(t[idx], y[idx])/batch_size
        return cost
    
# 편미분을 통한 기울기
# 행렬로 각 변수의 기울기 계산
def numerical_gradient_no_batch(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x와 똑같은 0행렬
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x+h) 계산
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)
        
        # f(x-h) 계산
        x[idx] = float(tmp_val) - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1-fxh2)/2*h
                
    return grad
    
# 편미분으로 기울기 산출
def numerical_gradient(f, x):    
    grad = np.zeros_like(x) # x와 똑같은 0행렬
    if x.ndim == 1:    
        return numerical_gradient_no_batch(f, x)
    else:
        for idx, x in enumerate(x):
            grad[idx] = numerical_gradient_no_batch(f, x)
        return grad
            
# 경사 하강법
def gradient_descent(f, x, lr=0.01, step_num=100):        
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x = x - learning_rate * grad
    return x

# 활성화 함수 softmax함수
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a    
    return y

# sigmoid 함수
def sigmoid(a):
    exp_a = np.exp(-a)
    y = 1/(1 + exp_a)
    return y

### 신경망

In [38]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):        
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        return y
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)        
        return cross_entropy_error_avg(y, t)        
    
    # 정확도
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    # x: 입력 데이터, t: 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        y = self.predict(x)
        
        # backward
        dy = (y -t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)
        
        return grads
        

In [39]:
# node 세팅
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape)

x = np.random.rand(1, 784) # 더미 입력 데이터 1장
t = np.random.rand(1, 10) # 더미 정답 레이블 1장

grads = net.numerical_gradient(x, t) # 테스트용 기울기 계산
print(grads['W1'].shape)

(784, 100)
(784, 100)


In [40]:
# data 세팅
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

print('x_train.shape :', x_train.shape)
print('t_train.shape :', t_train.shape)

train_size = x_train.shape[0]
batch_size = 10
# 0~ 59999에서 10개 random하게 추출
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

print('batch_mask :', batch_mask)
print('x_batch.shape :', x_batch.shape)
print('t_batch.shape :', t_batch.shape)

x_train.shape : (60000, 784)
t_train.shape : (60000, 10)
batch_mask : [12310 35149 49790 53204 44480 26901 43245 39663 26634 45150]
x_batch.shape : (10, 784)
t_batch.shape : (10, 10)


#### 구현부